In [4]:
import pandas as pd
import numpy as np

In [5]:
behavior_merged = pd.read_csv('preprocessed-news.csv')

In [6]:
behavior_merged.head()

,Unnamed: 0.1,Unnamed: 0,user,clicked_news,clicked_value,title,abstract,combined
0,0,0,U13740,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...
1,1,1,U10045,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...
2,2,2,U85394,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...
3,3,3,U78244,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...
4,4,4,U27024,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
import heapq

In [8]:
vectorizer = TfidfVectorizer(min_df = 0)

In [116]:
data_unique = behavior_merged.drop_duplicates(subset=['clicked_news'])

In [159]:
data_unique.head()

,Unnamed: 0.1,Unnamed: 0,user,clicked_news,clicked_value,title,abstract,combined
0,0,0,U13740,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...
3045,3045,3045,U13740,N42782,1,Three takeaways from Yankees' ALCS Game 5 vict...,The Yankees kept hope alive thanks to some imp...,three takeaway yankee alcs game victory astros...
3146,3146,3146,U13740,N34694,1,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,rosie odonnell barbara walter nt up speaking p...
3828,3828,3828,U13740,N45794,1,Four flight attendants were arrested in Miami'...,Four American Airlines flight attendants were ...,four flight attendant arrested miami s airport...
7111,7111,7111,U13740,N18445,1,Michigan sends breakup tweet to Notre Dame as ...,"Parting is such sweet sorrow, say the Wolverines.",michigan sends breakup tweet notre dame series...


In [189]:
user_interactions = data_unique.groupby('user').agg({'clicked_news': 'count', 'clicked_value': 'sum'}).reset_index()

In [190]:
user_interactions.head()

,user,clicked_news,clicked_value
0,U100,1,1
1,U10001,3,2
2,U10003,3,3
3,U10008,5,3
4,U10011,1,1


In [191]:
user_interactions[user_interactions['user'] == 'U13740']

,user,clicked_news,clicked_value
561,U13740,11,10


In [192]:
%%time
tfidf_matrix = vectorizer.fit_transform(data_unique['combined'])

CPU times: total: 1.08 s
Wall time: 1.08 s


In [193]:
tfidf_matrix

<48616x54775 sparse matrix of type '<class 'numpy.float64'>'
	with 1154151 stored elements in Compressed Sparse Row format>

In [194]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100)  # Adjust the number of components as needed

In [195]:
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)
tfidf_matrix_reduced.shape

(48616, 100)

In [126]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm

In [127]:
tfidf_matrix_sparse = csr_matrix(tfidf_matrix_reduced)

In [129]:
num_neighbors = 10
nbrs = NearestNeighbors(n_neighbors=num_neighbors, algorithm='brute', metric='cosine')
nbrs.fit(tfidf_matrix_reduced)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [ ]:
!pip install scikit-learn

In [29]:
tfidf_matrix_sparse.shape

(5706634, 10)

In [30]:
tfidf_matrix_dense = tfidf_matrix_sparse.toarray()

In [31]:
tfidf_matrix_dense

array([[ 0.0898752 , -0.00577011, -0.00075039, ..., -0.02234794,
        -0.01875708, -0.01683801],
       [ 0.0898752 , -0.00577011, -0.00075039, ..., -0.02234794,
        -0.01875708, -0.01683801],
       [ 0.0898752 , -0.00577011, -0.00075039, ..., -0.02234794,
        -0.01875708, -0.01683801],
       ...,
       [ 0.07062028,  0.0018471 ,  0.00348129, ..., -0.00729444,
         0.01468261, -0.01116064],
       [ 0.06828962,  0.00075246,  0.00092491, ..., -0.01097424,
        -0.00782517, -0.02083914],
       [ 0.03348772,  0.01623461, -0.00289927, ..., -0.02046035,
        -0.00671981,  0.00873379]])

In [199]:
from sklearn.metrics.pairwise import cosine_similarity

In [216]:
def recommend_diverse_articles(user_id, num_recommendations=10):
    # Filter user interactions
    user_interactions_filtered = user_interactions[user_interactions['user'] == user_id]
    user_article_ids = user_interactions_filtered['clicked_news'].tolist()
    
    # Calculate user profile based on the articles the user has interacted with
    user_profile = tfidf_matrix[user_article_ids].mean(axis=0)
    
    user_profile = user_profile.A.squeeze()

    
    # Calculate cosine similarity between user profile and all articles
    similarity_scores = cosine_similarity(user_profile.reshape(1, -1), tfidf_matrix)
    
    # Find indices of top N articles with highest similarity scores
    recommended_indices = similarity_scores.argsort()[0][::-1][:num_recommendations]
    
    # Return recommended articles
    recommended_articles = data_unique.iloc[recommended_indices]
    
    return recommended_articles


In [213]:
def recommend_diverse_articles(user_id, num_recommendations=5):
    # Filter user interactions
    user_interactions_filtered = data_unique[data_unique['user'] == user_id]
    user_article_ids = user_interactions_filtered['clicked_news'].tolist()
    
    
    # Calculate user profile based on the articles the user has interacted with
    user_profile = vectorizer.transform(user_interactions['combined']).mean(axis=0)
    
    # Reduce dimensionality of user profile
    user_profile_svd = svd.transform(user_profile)
    
    # Find nearest neighbors to user profile
    distances, indices = nbrs.kneighbors(user_profile_svd)
    
    # Get recommended article indices
    recommended_indices = indices.flatten()
    
    # Return recommended articles
    recommended_articles = data_unique.iloc[recommended_indices]
    
    return recommended_articles

In [217]:
user_id = 'U13740'

In [221]:
recommended_articles= recommend_diverse_articles(user_id)

In [223]:
print(recommended_articles['title'])


40341      'Habitual stowaway' arrested again at O'Hare, ...
5222135    'Serial Stowaway' Marilyn Hartman Held Without...
1104203    "Serial stowaway" arrested at Chicago airport ...
5586705    Serial stowaway held without bond for violatin...
5694315    The Latest: Snow causes 888 flight cancelation...
5576509    The Latest: Snow causes 888 flight cancelation...
3940901    Plane slides off runway at Chicago's O'Hare am...
5052888    The Latest: About 1,200 flights cancelled at O...
3936149    Dramatic Footage Shows Plane Skidding Off Runw...
5448816    Chicago Weather: Over 900 Flights Canceled At ...
Name: title, dtype: object


In [219]:
data_unique[data_unique['user'] == 'U13740']

,Unnamed: 0.1,Unnamed: 0,user,clicked_news,clicked_value,title,abstract,combined
0,0,0,U13740,N55189,1,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",wheel fortune guest delivers hilarious rail in...
3045,3045,3045,U13740,N42782,1,Three takeaways from Yankees' ALCS Game 5 vict...,The Yankees kept hope alive thanks to some imp...,three takeaway yankee alcs game victory astros...
3146,3146,3146,U13740,N34694,1,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,rosie odonnell barbara walter nt up speaking p...
3828,3828,3828,U13740,N45794,1,Four flight attendants were arrested in Miami'...,Four American Airlines flight attendants were ...,four flight attendant arrested miami s airport...
7111,7111,7111,U13740,N18445,1,Michigan sends breakup tweet to Notre Dame as ...,"Parting is such sweet sorrow, say the Wolverines.",michigan sends breakup tweet notre dame series...
7375,7375,7375,U13740,N63302,1,This Wedding Photo of a Canine Best Man Captur...,"When Mark Doublet made his dog, Marley, the be...",wedding photo canine best man capture deep dog...
7881,7881,7881,U13740,N10414,1,"Robert Evans, 'Chinatown' Producer and Paramou...","Robert Evans, the Paramount executive who prod...",robert evans chinatown producer paramount chie...
8390,8390,8390,U13740,N19347,1,Former US Senator Kay Hagan dead at 66,"Former U.S. Sen. Kay Hagan, a one-time Capitol...",former u senator kay hagan dead former us sen ...
9186,9186,9186,U13740,N31801,1,Joe Biden reportedly denied Communion at a Sou...,Joe Biden has a complicated history with the C...,joe biden reportedly denied communion south ca...
12393,12393,12393,U13740,N55689,1,"Charles Rogers, former Michigan State football...","Charles Rogers, the former Michigan State foot...",charles rogers former michigan state football ...
